In [1]:
%matplotlib inline

from pathlib import Path

import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.neighbors import NearestNeighbors, KNeighborsClassifier
import matplotlib.pylab as plt

## 7.1

In [2]:
#a

In [3]:
data=pd.DataFrame({'job':['IT', 'Stat', 'Other'], 'year':[1,1,1.1]})
data

,job,year
0,IT,1.0
1,Stat,1.0
2,Other,1.1


In [4]:
# 3개의 이진변수 변환을 위해 가변수 적용
job_3 = pd.get_dummies(data)
job_3

,year,job_IT,job_Other,job_Stat
0,1.0,1,0,0
1,1.0,0,0,1
2,1.1,0,1,0


In [5]:
# 두 개를 가변수 적용하면 나머지 하나는 자동적으로 결정됨
job_2 = pd.get_dummies(data, drop_first=True)
job_2

,year,job_Other,job_Stat
0,1.0,0,0
1,1.0,0,1
2,1.1,1,0


In [6]:
#b 잠재고객과 각 고객의 유클리드 거리 계산

In [7]:
c=job_3.iloc[1:]
p=job_3.iloc[:1]

In [8]:
knn = NearestNeighbors(n_neighbors=1)
knn.fit(c)
distances, indices = knn.kneighbors(p) # 거리구하기
print(distances) 

[[1.41421356]]


In [9]:
knn = NearestNeighbors(n_neighbors=2)
knn.fit(c)
distances, indices = knn.kneighbors(p) # 거리구하기
print(distances) 

[[1.41421356 1.41774469]]


In [10]:
#c k=1 일때 잠재고객을 코스 수강함 코스 수강하지 않음으로 분류

In [11]:
# 3개의 가변수
c3=job_3.loc[1:]
c3['course']=['NO', 'YES']
c3

C:\Users\Public\Documents\ESTsoft\CreatorTemp/ipykernel_23044/3666573112.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  c3['course']=['NO', 'YES']


,year,job_IT,job_Other,job_Stat,course
1,1.0,0,0,1,NO
2,1.1,0,1,0,YES


In [12]:
c3_o=job_3.loc[:0]
c3_o

,year,job_IT,job_Other,job_Stat
0,1.0,1,0,0


In [13]:
knn = NearestNeighbors(n_neighbors=1)
knn.fit(c3[['year', 'job_IT', 'job_Other','job_Stat']])
distances, indices = knn.kneighbors(c3_o)
print(c3.iloc[indices[0],:])

   year  job_IT  job_Other  job_Stat course
1   1.0       0          0         1     NO


In [14]:
# 가변수 2개
c2=job_2.loc[1:]
c2['course']=['NO', 'YES']

C:\Users\Public\Documents\ESTsoft\CreatorTemp/ipykernel_23044/3658196055.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  c2['course']=['NO', 'YES']


In [15]:
c2_o=job_2.loc[:0]
c2_o

,year,job_Other,job_Stat
0,1.0,0,0


In [16]:
knn = NearestNeighbors(n_neighbors=1)
knn.fit(c2[['year', 'job_Other','job_Stat']])
distances, indices = knn.kneighbors(c2_o)
print(c2.iloc[indices[0],:])

   year  job_Other  job_Stat course
1   1.0          0         1     NO


In [17]:
## 수강하지 않음으로 분류한다.

## 7.2

In [18]:
univ = pd.read_csv('UniversalBank.csv')

In [19]:
univ.columns

Index(['ID', 'Age', 'Experience', 'Income', 'ZIP Code', 'Family', 'CCAvg',
       'Education', 'Mortgage', 'Personal Loan', 'Securities Account',
       'CD Account', 'Online', 'CreditCard'],
      dtype='object')

In [20]:
univ=univ.drop(columns=['ID','ZIP Code'])
univ=pd.get_dummies(univ,columns=['Education'])
univ

,Age,Experience,Income,Family,CCAvg,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard,Education_1,Education_2,Education_3
0,25,1,49,4,1.6,0,0,1,0,0,0,1,0,0
1,45,19,34,3,1.5,0,0,1,0,0,0,1,0,0
2,39,15,11,1,1.0,0,0,0,0,0,0,1,0,0
3,35,9,100,1,2.7,0,0,0,0,0,0,0,1,0
4,35,8,45,4,1.0,0,0,0,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,29,3,40,1,1.9,0,0,0,0,1,0,0,0,1
4996,30,4,15,4,0.4,85,0,0,0,1,0,1,0,0
4997,63,39,24,2,0.3,0,0,0,0,0,0,0,0,1
4998,65,40,49,3,0.5,0,0,0,0,1,0,0,1,0


In [21]:
trainData, validData = train_test_split(univ, test_size=0.4, random_state=26) # 데이터 분할

In [22]:
#a

In [23]:
new_c=pd.DataFrame([{'Age':40, 'Experience':10, 'Income':84, 'Family':2, 'CCAvg':2, 
 'Education_1':0, 'Education_2':1, 'Education_3':0,
'Mortgage':0, 'Securities Account':0, 'CD Account':0, 'Online':1, 'CreditCard':1}])
new_c

,Age,Experience,Income,Family,CCAvg,Education_1,Education_2,Education_3,Mortgage,Securities Account,CD Account,Online,CreditCard
0,40,10,84,2,2,0,1,0,0,0,0,1,1


In [24]:
scaler = preprocessing.StandardScaler()

scaler.fit(trainData[[ 'Age', 'Experience', 'Income', 'Family', 'CCAvg',
       'Education_1', 'Education_2', 'Education_3', 'Mortgage', 'Securities Account',
       'CD Account', 'Online', 'CreditCard']])  # Note the use of an array of column names

# Transform the full dataset
univNorm = pd.concat([pd.DataFrame(scaler.transform(univ[['Age', 'Experience', 'Income', 'Family', 'CCAvg',
       'Education_1', 'Education_2', 'Education_3', 'Mortgage', 'Securities Account',
       'CD Account', 'Online', 'CreditCard']]), 
                                    columns=['zAge', 'zExperience', 'zIncome', 'zFamily', 'zCCAvg',
       'zEducation_1', 'zEducation_2', 'zEducation_3', 'zMortgage', 'zSecurities Account',
       'zCD Account', 'zOnline', 'zCreditCard']),
                       univ[[ 'Personal Loan']]], axis=1)

trainNorm = univNorm.iloc[trainData.index]
validNorm = univNorm.iloc[validData.index]
newC_Norm = pd.DataFrame(scaler.transform(new_c), 
                                 columns=['zAge', 'zExperience', 'zIncome', 'zFamily', 'zCCAvg',
       'zEducation_1', 'zEducation_2', 'zEducation_3', 'zMortgage', 'zSecurities Account',
       'zCD Account', 'zOnline', 'zCreditCard'])

In [25]:
columns=['zAge', 'zExperience', 'zIncome', 'zFamily', 'zCCAvg',
       'zEducation_1', 'zEducation_2', 'zEducation_3', 'zMortgage', 'zSecurities Account',
       'zCD Account', 'zOnline', 'zCreditCard']

In [26]:
knn = NearestNeighbors(n_neighbors=1)
knn.fit(trainNorm[columns])
distances, indices = knn.kneighbors(newC_Norm)
print(trainNorm.iloc[indices[0],:])  ## 대출수락되지 않음

          zAge  zExperience   zIncome   zFamily    zCCAvg  zEducation_1  \
4407 -0.719804    -0.613601 -0.042428 -0.340587 -0.122566     -0.856799   

      zEducation_2  zEducation_3  zMortgage  zSecurities Account  zCD Account  \
4407      1.587806     -0.643242  -0.547625            -0.346151    -0.248891   

       zOnline  zCreditCard  Personal Loan  
4407  0.806328     1.549632              0  


In [27]:
#b

In [28]:
train_X = trainNorm[columns]
train_y = trainNorm['Personal Loan']
valid_X = validNorm[columns]
valid_y = validNorm['Personal Loan']

# Train a classifier for different values of k
results = []
for k in range(1, 15):
    knn = KNeighborsClassifier(n_neighbors=k).fit(train_X, train_y)
    results.append({
        'k': k,
        'accuracy': accuracy_score(valid_y, knn.predict(valid_X))
    })

# Convert results to a pandas data frame
results = pd.DataFrame(results)
print(results)

     k  accuracy
0    1    0.9550
1    2    0.9460
2    3    0.9555
3    4    0.9445
4    5    0.9525
5    6    0.9445
6    7    0.9495
7    8    0.9425
8    9    0.9460
9   10    0.9430
10  11    0.9450
11  12    0.9390
12  13    0.9405
13  14    0.9350


In [29]:
#c 정오행렬표 만들기

In [30]:
import matplotlib.pylab as plt

from dmba import regressionSummary, classificationSummary, liftChart, gainsChart

In [31]:
knn = KNeighborsClassifier(n_neighbors=3).fit(train_X, train_y)
classificationSummary(valid_y, knn.predict(valid_X), class_names=['대출 미수락','대출 수락'])

Confusion Matrix (Accuracy 0.9555)

       Prediction
Actual 대출 미수락  대출 수락
대출 미수락   1779      6
 대출 수락     83    132


In [32]:
#d 최적의 k 사용해 고객들을 분류하기

In [33]:
knn = NearestNeighbors(n_neighbors=3)
knn.fit(trainNorm[columns])
distances, indices = knn.kneighbors(newC_Norm)
print(trainNorm.iloc[indices[0],:]) 

          zAge  zExperience   zIncome   zFamily    zCCAvg  zEducation_1  \
4407 -0.719804    -0.613601 -0.042428 -0.340587 -0.122566     -0.856799   
3398 -0.459477    -0.526770 -0.240212 -0.340587  0.279333     -0.856799   
1630 -0.372701    -0.266279  0.572898 -0.340587 -0.065152     -0.856799   

      zEducation_2  zEducation_3  zMortgage  zSecurities Account  zCD Account  \
4407      1.587806     -0.643242  -0.547625            -0.346151    -0.248891   
3398      1.587806     -0.643242  -0.547625            -0.346151    -0.248891   
1630      1.587806     -0.643242  -0.547625            -0.346151    -0.248891   

       zOnline  zCreditCard  Personal Loan  
4407  0.806328     1.549632              0  
3398  0.806328     1.549632              0  
1630  0.806328     1.549632              0  


In [34]:
##e 0.5 0.3 0.2 나누기 

In [35]:
trainNorm, temp = train_test_split(univ, test_size= 0.5, random_state=0)
validNorm, testNorm = train_test_split(temp, test_size= 0.4, random_state=1)

In [36]:
train_X = trainNorm.drop(columns=['Personal Loan'])
train_y = trainNorm['Personal Loan']

In [37]:
knn = KNeighborsClassifier(n_neighbors=3).fit(train_X, train_y)

In [38]:
classificationSummary(train_y, knn.predict(train_X), class_names=['대출 미수락','대출 수락'])

Confusion Matrix (Accuracy 0.9508)

       Prediction
Actual 대출 미수락  대출 수락
대출 미수락   2225     37
 대출 수락     86    152


In [41]:
valid_X = validNorm.drop(columns=['Personal Loan'])
valid_y = validNorm['Personal Loan']
knn = KNeighborsClassifier(n_neighbors=3).fit(valid_X, valid_y)
classificationSummary(valid_y, knn.predict(valid_X), class_names=['대출 미수락','대출 수락'])

Confusion Matrix (Accuracy 0.9520)

       Prediction
Actual 대출 미수락  대출 수락
대출 미수락   1337     24
 대출 수락     48     91


In [42]:
test_X = testNorm.drop(columns=['Personal Loan'])
test_y = testNorm['Personal Loan']
knn = KNeighborsClassifier(n_neighbors=3).fit(test_X, test_y)
classificationSummary(test_y, knn.predict(test_X), class_names=['대출 미수락','대출 수락'])

Confusion Matrix (Accuracy 0.9380)

       Prediction
Actual 대출 미수락  대출 수락
대출 미수락    881     16
 대출 수락     46     57


## 3

In [43]:
boston = pd.read_csv('BostonHousing.csv')
boston.columns

Index(['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX',
       'PTRATIO', 'LSTAT', 'MEDV', 'CAT. MEDV'],
      dtype='object')

In [44]:
boston=boston.drop(columns=['CAT. MEDV'])

In [49]:
boston.MEDV = boston.MEDV.astype(int)
boston.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 13 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   CRIM     506 non-null    float64
 1   ZN       506 non-null    float64
 2   INDUS    506 non-null    float64
 3   CHAS     506 non-null    int64  
 4   NOX      506 non-null    float64
 5   RM       506 non-null    float64
 6   AGE      506 non-null    float64
 7   DIS      506 non-null    float64
 8   RAD      506 non-null    int64  
 9   TAX      506 non-null    int64  
 10  PTRATIO  506 non-null    float64
 11  LSTAT    506 non-null    float64
 12  MEDV     506 non-null    int32  
dtypes: float64(9), int32(1), int64(3)
memory usage: 49.5 KB


In [53]:
# MEDV 삭제
x_boston= boston.drop('MEDV', axis=1)

# MEDV 추출
m_boston=boston['MEDV']

In [54]:
# 데이터 분할
trainData, validData = train_test_split(boston, test_size=0.4, random_state=26) # 데이터 분할

In [55]:
boston.columns

Index(['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX',
       'PTRATIO', 'LSTAT', 'MEDV'],
      dtype='object')

In [56]:
# 데이터 정규화

scaler = preprocessing.StandardScaler()
train_X = trainData.iloc[:,:12]
train_y = trainData.iloc[:,12]

scaler.fit(train_X)  

# Transform the full dataset
train_y.index=list(range(303))
tNorm=pd.concat([pd.DataFrame(scaler.transform(train_X),
                                   columns=['zCRIM', 'zZN','zINDUS','zCHAS','zNOX','zRM','zAGE','zDIS',
                                  'zRAD','zTAX','zPTRATIO','zLSTAT']),
                                   train_y],axis=1)

tNorm

,zCRIM,zZN,zINDUS,zCHAS,zNOX,zRM,zAGE,zDIS,zRAD,zTAX,zPTRATIO,zLSTAT,MEDV
0,-0.432821,-0.428676,-0.181554,-0.251312,-0.074298,-0.599430,0.104857,-0.597442,-0.399488,0.154536,-0.334603,0.361212,20
1,-0.443598,-0.428676,-1.000951,-0.251312,-0.978185,0.123438,-0.822970,0.543440,-0.747651,-0.963638,0.001108,-0.427283,23
2,0.495733,-0.428676,1.019338,-0.251312,1.502892,-0.412072,0.925484,-0.896256,1.689491,1.568875,0.816406,0.406464,14
3,1.814624,-0.428676,1.019338,-0.251312,1.336872,-1.431358,1.028987,-1.118544,1.689491,1.568875,0.816406,2.478491,7
4,-0.419524,-0.428676,-0.641723,-0.251312,-0.987408,-0.378515,0.563225,0.994461,-0.747651,-1.048257,-0.286644,0.831566,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,0.563558,-0.428676,1.019338,-0.251312,1.456775,0.162588,1.036380,-0.742413,1.689491,1.568875,0.816406,0.901502,13
299,0.417787,-0.428676,1.019338,-0.251312,0.266965,0.173773,0.167698,-0.734780,1.689491,1.568875,0.816406,-0.096801,16
300,-0.380336,0.502928,-1.078141,-0.251312,0.848035,2.932419,0.785016,-0.691116,-0.515542,-0.860887,-2.636622,-0.936033,48
301,-0.396250,-0.428676,-0.197883,-0.251312,-0.101968,0.113651,-0.113238,-0.074556,-0.631596,-0.619120,-0.046851,-0.325807,23


In [57]:
# 검증 데이터

valid_X = validData.iloc[:,:12]
valid_y = validData.iloc[:,12]

scaler.fit(valid_X)  

# (303, 13) (203, 13)
valid_y.index=list(range(203))
vNorm=pd.concat([pd.DataFrame(scaler.transform(valid_X),
                                   columns=['zCRIM', 'zZN','zINDUS','zCHAS','zNOX','zRM','zAGE','zDIS',
                                  'zRAD','zTAX','zPTRATIO','zLSTAT']),
                                   valid_y],axis=1)

vNorm

,zCRIM,zZN,zINDUS,zCHAS,zNOX,zRM,zAGE,zDIS,zRAD,zTAX,zPTRATIO,zLSTAT,MEDV
0,-0.364884,-0.573166,1.551830,-0.302321,0.554631,0.289862,1.114725,-0.832355,-0.647525,0.151032,1.235448,0.010247,19
1,-0.390814,1.194813,-1.075627,-0.302321,-0.930360,0.765698,-0.846927,-0.078449,-0.534110,-0.074917,-1.404973,-0.849096,32
2,-0.311181,-0.573166,1.551830,-0.302321,0.554631,-0.915881,0.972182,-0.894921,-0.647525,0.151032,1.235448,0.843461,14
3,-0.260896,-0.573166,-0.406302,-0.302321,-0.128306,-0.456201,0.541162,0.012465,-0.647525,-0.602132,1.147434,2.203603,13
4,-0.388960,-0.573166,-0.716755,-0.302321,-0.438010,-0.616281,-0.157973,-0.263845,-0.534110,-0.764352,0.355308,-0.162493,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...
198,-0.373623,-0.573166,-0.057399,-0.302321,-0.517421,-0.701461,0.225533,0.177186,-0.647525,-0.775939,0.091266,0.802816,22
199,-0.397338,0.801929,-1.349054,-0.302321,-0.890654,1.439798,-0.568631,1.388329,-0.987769,-0.735384,-1.272952,-1.021835,32
200,-0.386972,-0.082061,-0.444753,-0.302321,-0.239482,-0.369552,0.571707,1.022142,-0.534110,-0.578958,-1.404973,0.107504,18
201,-0.389630,1.194813,-1.075627,-0.302321,-0.930360,1.013896,-1.512124,1.136395,-0.534110,-0.074917,-1.404973,-1.078447,37


In [58]:
# Retrain with full dataset
# 검증데이터로 예측

# k=1 - 5
valid_X = validData.iloc[:,:12]
valid_y = validData.iloc[:,12]

for i in range(1,6):
    knn = KNeighborsClassifier(n_neighbors=i).fit(valid_X, valid_y)
    distances, indices = knn.kneighbors(valid_X)
    print("(k=%d)" %i)
    print(knn.predict(valid_X))
    print()
    print()

(k=1)
[19 32 14 13 20 30 23 22 20 13 14 17 16 19 20 14 20 21 36  8 22 48 22 27
 16 23 21 21 30 20 23  8 27 35 20 19 10 50 26 13 34 46 20 24 24 45 14 15
 24 36 19 19 18 21 28 18 25 23 20 30 21 23 19 18 10 15 23 22 19 50 20 18
  8 16 21 33 23 19 34 23 30 10 20 19 22 11 19 43 20 20 13 13 11 24 23 25
 22 23 22 31 11 25 25 17 24  7 15 32 37 24 29 22 18 20 20 22 17 20 22 10
 36 50 18 27 33 48 12 16 19 25 22 30 26 24 25 14 19 34 22 14 22 16 23 20
 50 29  7 19 23 32 35 25 16 16 22 18 36 22 22 13 14 24 17 12 28 14 13 22
 31 28 19 22 32 46  8 27 11 19 19 21 17 21 17 13  5 21 10 12 10 11 14 16
 12 15 33 21 21 19 22 32 18 37 18]


(k=2)
[18 32 14 13 20 30 23 21 20 13 12 17 16 19 20 14 19 21 29  7 22 27 21 27
 12 23 19 19 30 19 22  8 23 35 14 19 10 30 24 12 32 35 19 24 24 24 12 15
 22 30 19 18 17 21 18 16 25 19 20 27 21 22 11 18  7 15 20 22 19 27 20 18
  8 16 20 30 23 19 32 22 30  5 20 19 21 11 19 36 13 20 13  5 11 22 20 25
 22 23 22 31 11 25 19 17 22  7 15 24 36 20 29 22 13 19 20 20 17 19 19 10
 3

In [59]:
results=[]

for k in range(1, 15):
    knn = KNeighborsClassifier(n_neighbors=k).fit(train_X, train_y)
    results.append({
        'k': k,
        'accuracy': accuracy_score(valid_y, knn.predict(valid_X))
    })

# Convert results to a pandas data frame
results = pd.DataFrame(results)
print(results)

     k  accuracy
0    1  0.137931
1    2  0.142857
2    3  0.103448
3    4  0.118227
4    5  0.123153
5    6  0.118227
6    7  0.123153
7    8  0.128079
8    9  0.133005
9   10  0.123153
10  11  0.098522
11  12  0.108374
12  13  0.103448
13  14  0.128079


k = 2 일때 정확도가 가장 높음

In [61]:
#b

In [62]:
newboston = pd.DataFrame([{'CRIM': 0.2, 'ZN':0, 'INDUS':7, 'CHAS':0, 'NOX':0.538, 'RM':6,
                          'AGE':62, 'DIS':4.7, 'RAD':4, 'TAX':307, 'PTRATIO':21, 'LSTAT':10}])
newboston

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,LSTAT
0,0.2,0,7,0,0.538,6,62,4.7,4,307,21,10


In [63]:
knn = NearestNeighbors(n_neighbors=2)
knn.fit(train_X)

#새로운 트랙 생성 및 정규화
train_X1=train_X
train_X1.loc["new"]=[0.2,0,7,0,0.538,6,62,4.7,4,307,21,10]

scaler = preprocessing.StandardScaler()
scaler.fit(train_X1)

newNorm=pd.concat([pd.DataFrame(scaler.transform(train_X1),columns=validData.columns[:12]),
                     valid_y],axis=1)

newNorm=vNorm.iloc[202][:12] 
newNorm=newNorm.to_frame().transpose()
distances,indices=knn.kneighbors(newNorm)

j=0
sse=0
for neighbor in indices.tolist(): 
    predict=0 
    for i in neighbor: 
        predict+=train_y[i] 
    predict=predict/2 
    
print(predict) 

36.0


In [64]:
## c

In [66]:
for k in range(2,3):
    knn = NearestNeighbors(n_neighbors=k) 
    knn.fit(train_X)
    
    distances, indices = knn.kneighbors(valid_X) 
    print(k)
    
    j=0
    sse=0
    for neighbor in indices.tolist(): 
        predict=0 
        for i in neighbor: 
            predict+=train_y[i] 
        predict=predict/k 
        sse+=(predict-valid_y.iloc[j])**2
        j+=1
    mse=sse/202 
    print(k, '\n',mse)

2
2 
 42.14108910891089


In [67]:
#d

정확도가 높은 기준으로 k를 선정했기 때문에 오류가 낙관적

In [68]:
#e

수천 개의 새로운 트랙에 대한 y값을 예측할때 근접 이웃을 찾는 시간이 길어질 수 있고,
예측변수의 개수가 많아 훈련 세트의 개수도 증가한다는 단점이 있다.